In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# config: pollutants + mapping to sensors
POLLUTANTS = ["CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]
GT_TO_SENSOR = {
    "CO(GT)": "PT08.S1(CO)",
    "C6H6(GT)": "PT08.S2(NMHC)",
    "NOx(GT)": "PT08.S3(NOx)",
    "NO2(GT)": "PT08.S4(NO2)",
}


# ---------- 0) I/O utils ----------
def get_reports_dir():
    """Return reports directory under src/ or working directory."""
    # In Jupyter, __file__ does NOT exist → fallback to cwd
    try:
        base_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        base_dir = os.getcwd()

    reports = os.path.join(base_dir, "reports")
    os.makedirs(reports, exist_ok=True)
    return reports

#  1) Load + minimal clean
def load_and_clean_data(path="air+quality/AirQualityUCI.xlsx"):
    df = pd.read_excel(path)
    
    # remove trailing spaces in column names
    df.columns = df.columns.str.strip()
    
    # Merge Date + Time
    df["DateTime"] = pd.to_datetime(
        df["Date"].astype(str) + " " + df["Time"].astype(str),
        errors="coerce",
        dayfirst=True,
    )
    df.drop(["Date", "Time"], axis=1, inplace=True)
    
    # replace invalid sensor/sentinals with NaN
    df.replace(-200, np.nan, inplace=True)
    
    # drop rows if DateTime could not be read
    df = df.dropna(subset=["DateTime"]).sort_values("DateTime").reset_index(drop=True)
    
    # add temporal features to use for anomaly context
    df["hour"] = df["DateTime"].dt.hour
    df["weekday"] = df["DateTime"].dt.day_name()
    df["month"] = df["DateTime"].dt.month
    return df


# 2) Residual-based anomalies (generic) 
def detect_residual_anomalies(df, col, win=12, k_sigma=3.5):
    """
    Detect anomalies based on rolling median and Median Absolute Deviation (MAD).
    Any point deviating more than k_sigma * sigma_robust from rolling baseline
    is flagged as anomalous.
    """
    df = df.copy()
    short = col.split("(")[0]  # e.g. CO(GT) -> "CO"
    
    # names for new columns
    med_col = f"{col}_median"
    resid_col = f"{col}_resid"
    flag_col = f"is_anom_resid_{short}"

    # rolling median baseline
    df[med_col] = df[col].rolling(win, center=True,
                                  min_periods=max(3, win // 2)).median()
    
    # compute residual from rolling baseline
    df[resid_col] = df[col] - df[med_col]

    # compte MAD-based robust sigma
    mad = np.nanmedian(np.abs(df[resid_col] - np.nanmedian(df[resid_col])))
    sigma = 1.4826 * mad
    
    # threshold to detect anomalies
    thr = k_sigma * sigma if np.isfinite(sigma) and sigma > 0 else np.nan
    
    # flag anomalies if threshold exists
    df[flag_col] = np.abs(df[resid_col]) > thr if np.isfinite(thr) else False
    
    return df, flag_col, thr


# 3) Multivariate Isolation Forest 
def detect_isolation_forest(df, features=None, contamination=0.05, random_state=42):
    if features is None:
        # by default use pollutants + meteorological variables
        features = POLLUTANTS + ["T", "RH", "AH"]

    # drop rows missing predictors
    X = df[features].dropna()
    
    # scale features 
    scaler = StandardScaler()
    X_sc = scaler.fit_transform(X.values)

    # fit isolation forest
    iso = IsolationForest(contamination=contamination, random_state=random_state)
    y_pred = iso.fit_predict(X_sc)  # -1 = anomaly

    df_iso = df.copy()
    df_iso["is_anom_iso"] = False
    df_iso.loc[X.index, "is_anom_iso"] = y_pred == -1
    return df_iso, features


# 4) Sensor–truth consistency (generic)
def sensor_truth_mismatch(df, truth_col, sensor_col, z_thr=3.0):
    """
    For a given GT pollutant and its sensor column, flag rows where
    their standardized values differ strongly.
    Writes is_mismatch_<short> column.
    """
    df = df.copy()
    short = truth_col.split("(")[0]
    flag_col = f"is_mismatch_{short}"

    # if sensor column missing, create "all False" flag
    if sensor_col not in df.columns:
        df[flag_col] = False
        return df, flag_col

    # extract valid rows
    sub = df[[truth_col, sensor_col]].dropna()
    if sub.empty:
        df[flag_col] = False
        return df, flag_col

    # z-standardise each series manually
    ztruth = (sub[truth_col] - sub[truth_col].mean()) / sub[truth_col].std(ddof=0)
    zsens = (sub[sensor_col] - sub[sensor_col].mean()) / sub[sensor_col].std(ddof=0)
    
    # absolute standard deviation difference between truth and sensor
    delta = (ztruth - zsens).abs()
    
    # points where difference > threshold
    mismatch_idx = sub.index[delta > z_thr]

    df[flag_col] = False
    df.loc[mismatch_idx, flag_col] = True
    return df, flag_col


# 5) Plotting helpers (scatter-only versions)
def plot_ts_with_anomalies(df, col, resid_flag_col, reports_dir):
    """
    Scatter plot for pollutant time-series with anomaly markers.
    """
    short = col.split("(")[0]
    plt.figure(figsize=(14, 5))

    # Base scatter (all points)
    plt.scatter(df["DateTime"], df[col], s=6, alpha=0.5, label=col, color="steelblue")

    # Residual anomalies
    if resid_flag_col in df.columns and df[resid_flag_col].any():
        mask = df[resid_flag_col]
        plt.scatter(df.loc[mask, "DateTime"], df.loc[mask, col],
                    s=20, color="red", label="Residual anomaly")

    # Isolation Forest anomalies
    if "is_anom_iso" in df.columns and df["is_anom_iso"].any():
        mask = df["is_anom_iso"]
        plt.scatter(df.loc[mask, "DateTime"], df.loc[mask, col],
                    s=20, color="orange", alpha=0.8, label="IsolationForest anomaly")

    plt.title(f"{col} with Anomalies (Scatter Plot)")
    plt.xlabel("Date")
    plt.ylabel(col)
    plt.legend()
    plt.tight_layout()
    fname = f"fig_{short.lower()}_anomalies.png"
    plt.savefig(os.path.join(reports_dir, fname), dpi=200)
    plt.close()


def plot_sensor_truth(df, truth_col, sensor_col, mismatch_flag_col, reports_dir):
    """
    Scatter plot version of sensor vs truth overlay.
    """
    if sensor_col not in df.columns:
        return

    short = truth_col.split("(")[0]
    plt.figure(figsize=(14, 4))

    # Scatter truth and sensor
    plt.scatter(df["DateTime"], df[truth_col],
                s=8, alpha=0.6, label=truth_col, color="dodgerblue")

    plt.scatter(df["DateTime"], df[sensor_col],
                s=8, alpha=0.4, label=sensor_col, color="orange")

    # Mismatch flags
    if mismatch_flag_col in df.columns and df[mismatch_flag_col].any():
        m = df[mismatch_flag_col]
        plt.scatter(df.loc[m, "DateTime"], df.loc[m, truth_col],
                    s=20, color="purple", label="Sensor–truth mismatch")

    plt.title(f"{short}: Truth vs Sensor (Scatter Plot)")
    plt.xlabel("Date")
    plt.legend()
    plt.tight_layout()
    fname = f"fig_{short.lower()}_truth_vs_sensor.png"
    plt.savefig(os.path.join(reports_dir, fname), dpi=200)
    plt.close()


def plot_anomaly_context(df, reports_dir):
    # Union over all residual flags + iso
    resid_flags = [c for c in df.columns if c.startswith("is_anom_resid_")]
    union = df.get("is_anom_iso", False)
    for c in resid_flags:
        union = union | df[c]
    anom = df.loc[union].copy()
    if anom.empty:
        return

    # By weekday
    ax = anom["weekday"].value_counts().reindex(
        ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    ).plot(kind="bar", figsize=(6, 4))
    ax.set_title("Anomalies by Weekday")
    ax.set_ylabel("Count")
    plt.tight_layout()
    plt.savefig(os.path.join(reports_dir, "fig_anom_by_weekday.png"), dpi=200)
    plt.close()

    # By hour
    ax = anom["hour"].value_counts().sort_index().plot(kind="bar", figsize=(8, 4))
    ax.set_title("Anomalies by Hour")
    ax.set_xlabel("Hour")
    ax.set_ylabel("Count")
    plt.tight_layout()
    plt.savefig(os.path.join(reports_dir, "fig_anom_by_hour.png"), dpi=200)
    plt.close()


# 6) MAIN 
if __name__ == "__main__":
    reports = get_reports_dir()

    df = load_and_clean_data("air+quality/AirQualityUCI.xlsx")

    # A) residual anomalies per pollutant
    resid_flag_cols = []
    for col in POLLUTANTS:
        if col not in df.columns:
            continue
        df, flag_col, thr_used = detect_residual_anomalies(df, col=col, win=12, k_sigma=3.5)
        resid_flag_cols.append(flag_col)
        print(f"[Residual] {col}: threshold ≈ {thr_used:.3f}" if np.isfinite(thr_used)
              else f"[Residual] {col}: threshold not computed")
    
    # multivariate IF
    df, used_feats = detect_isolation_forest(df,
                                             features=POLLUTANTS + ["T", "RH", "AH"],
                                             contamination=0.05)

    # sensor–truth mismatch for each pollutant with a sensor
    mismatch_flag_cols = []
    for gt, sensor in GT_TO_SENSOR.items():
        if gt in df.columns and sensor in df.columns:
            df, flag_col = sensor_truth_mismatch(df, truth_col=gt, sensor_col=sensor, z_thr=3.0)
            mismatch_flag_cols.append(flag_col)
        elif gt in df.columns:
            
            # still create column (all False) for consistency
            df[gt] = df[gt]
            
    # save CSV with flags
    out_cols = ["DateTime"] + POLLUTANTS + ["T", "RH", "AH", "hour", "weekday", "month"]
    out_cols += resid_flag_cols + ["is_anom_iso"] + mismatch_flag_cols
    out_cols = [c for c in out_cols if c in df.columns]
    df[out_cols].to_csv(os.path.join(reports, "anomalies_summary.csv"), index=False)

    # E) plots per pollutant
    for col, resid_flag in zip(POLLUTANTS, resid_flag_cols):
        if col in df.columns:
            plot_ts_with_anomalies(df, col, resid_flag, reports)
            sensor_col = GT_TO_SENSOR.get(col)
            if sensor_col:
                mismatch_flag_col = f"is_mismatch_{col.split('(')[0]}"
                plot_sensor_truth(df, col, sensor_col, mismatch_flag_col, reports)

    # context plots
    plot_anomaly_context(df, reports)

    # quick summary block
    total_iso = int(df["is_anom_iso"].sum())
    total_resid = sum(int(df[c].sum()) for c in resid_flag_cols)
    print(f"[Summary] IsolationForest anomalies: {total_iso}")
    print(f"[Summary] Residual anomalies (sum over pollutants): {total_resid}")
    print(f"Saved anomalies_summary.csv and figures under {reports}")


[Residual] CO(GT): threshold ≈ 2.076
[Residual] C6H6(GT): threshold ≈ 9.050
[Residual] NOx(GT): threshold ≈ 207.564
[Residual] NO2(GT): threshold ≈ 70.053
[Summary] IsolationForest anomalies: 131
[Summary] Residual anomalies (sum over pollutants): 594
Saved anomalies_summary.csv and figures under /Users/aayushveturi/Desktop/COMP9417/COMP9417_GroupProject/src/reports
